# Linear combination of ECD-rotation circuits

A fairly comprehensive use of optimizers based on `NumPy` objects to present qubit-qumode circuit(s) for a given qumode operator.

## Prerequisite

Installation cells for Google Colab users.

In [1]:
!pip install qutip
!pip install scipy
!pip install scikit-quant

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 89.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.7 MB/s eta 0:00:00
  Created wheel for scikit-quant: filename=scikit_quant-0.8.2-py3-none-any.whl size=21494 sha256=3e13929b769d9acbc8621a057f586340506406621e631c84e177d38ff22f85cd
  Stored in directory: /root/.cache/pip/wheels/f4/d4/a9/76f84a991c0bf8a333dd08a89f3089a1031b859f50cfbb6c37
  Created wheel for SQCommon: file

Import libaries.

In [2]:
import numpy as np
import qutip as qt
import skquant.opt as skopt
import scipy.optimize as sciopt

from scipy.linalg import expm
from functools import partial
from itertools import combinations

In [3]:
import sys

In [4]:
import matplotlib.pyplot as plt

## Basics

In [5]:
def np_arr_string(A):
    return np.array2string(A, separator=',')

In [6]:
def np_arr_append(beta_mag, beta_arg, theta, phi):
    beta_mag = np.append(beta_mag, np.random.uniform(0, 3))
    beta_arg = np.append(beta_arg, np.random.uniform(0, np.pi))
    theta = np.append(theta, np.random.uniform(0, np.pi))
    phi = np.append(phi, np.random.uniform(0, np.pi))
    return beta_mag, beta_arg, theta, phi

In [7]:
def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [8]:
def get_r_theta_np(cvec):
    cvec = np.array(cvec)
    r = np.abs(cvec)
    theta = np.angle(cvec)
    theta = np.where(theta < 0, theta + 2*np.pi, theta)
    return r, theta

## Ansatz

Elementary operator matrices.

In [9]:
def identity_np(N):
    return np.eye(N)

def basis_np(N, j):
    V = np.zeros(N)
    V[j] = 1.0
    return V

def destroy_np(N):
    op = np.diag(np.sqrt(np.arange(1, N)), k=1)
    return op

def create_np(N):
    return np.conj(destroy_np(N)).T

def sigmax_np():
    return np.array([[0 + 1j*0, 1], [1, 0]])

def sigmay_np():
    return np.array([[0, -1j], [1j, 0]])

def qproj00_np():
    return np.outer( basis_np(2, 0), basis_np(2, 0) )

def qproj11_np():
    return np.outer( basis_np(2, 1), basis_np(2, 1) )

def qproj01_np():
    return np.outer( basis_np(2, 0), basis_np(2, 1) )

def qproj10_np():
    return np.outer( basis_np(2, 1), basis_np(2, 0) )

def hadamard_np():
    return (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])

In [10]:
def displace_np(N, alpha):
    """
    D (alpha) = exp( alpha b! - alpha* b ).

    Arguments:
    alpha -- displacement parameter
    N -- Fock cutoff
    """
    gen = ( alpha * create_np(N) )
    gen -= ( np.conj(alpha) * destroy_np(N) )

    return expm(gen)

In [11]:
def rotation_np(theta, phi):
    """
    R (theta, phi) = exp[ −i (theta/2) ( X cos(phi) + Y sin(phi) ) ].

    Arguments:
    theta, phi: rotation parameters
    """
    gen = ( sigmax_np() * np.cos(phi) )
    gen += ( sigmay_np() * np.sin(phi) )

    H = -1j * (theta / 2) * gen

    return expm(H)

ECD-rotation operator matrix is defined as

$$ U (\beta, \theta, \phi)
= ECD (\beta) \: \big[ R (\theta, \phi) \otimes I \big]
= \begin{bmatrix}
e^{i \: (\phi - \pi/2)} \: \sin ( \theta/2 ) \: D^\dagger (\beta/2) &
\cos ( \theta/2 ) \: D^\dagger (\beta/2)
\\
\cos ( \theta/2 ) \: D (\beta/2) &
- e^{- i \: (\phi - \pi/2)} \: \sin ( \theta/2 ) \: D (\beta/2)
\end{bmatrix},
$$
where the operators are

\begin{align*}
R (\theta, \phi)
&= e^{ - i (\theta / 2) \big[ \cos(\phi) X + \sin(\phi) Y \big] },
\\
ECD (\beta)
&= |1 \rangle \langle 0| \otimes D (\beta / 2)  
+ |0 \rangle \langle 1| \otimes D (-\beta / 2),
\\
D (\alpha)
&= e^{ \alpha a^\dagger - \alpha^* a }.
\end{align*}

In [12]:
def ecd_rot_op_np(beta, theta, phi, nfock):
    """
    O = ECD(beta) (R (theta, phi) x I ).

    Reference DOI: 10.1038/s41567-022-01776-9.

    Arguments:
    beta -- ECD parameter
    theta, phi -- rotation parameters
    nfock -- number of qumode levels
    """
    ECD = np.kron(qproj10_np(), displace_np(nfock, beta/2))
    ECD += np.kron(qproj01_np(), displace_np(nfock, -beta/2))
    R = np.kron(rotation_np(theta, phi), identity_np(nfock))
    A = np.matmul(ECD, R)

    return A

In [13]:
def ecd_ansatz_np(beta, theta, phi, nfock):
    """
    ECD-rotation blocks.

    Arguments:
    beta -- Nb ECD parameters
    theta, phi -- Nb rotation parameters
    nfock -- number of qumode levels
    """
    # Check
    if beta.shape != theta.shape:
        raise ValueError("Lengths of beta and theta do not match.")
    if beta.shape != phi.shape:
        raise ValueError("Lengths of beta and phi do not match.")

    # Initialize
    ndepth = beta.shape[0]
    uni = ecd_rot_op_np(beta[0], theta[0], phi[0], nfock)

    # Check
    if ndepth == 1:
        return uni

    # Loop through blocks
    for j in range(1, ndepth):
        new_uni = ecd_rot_op_np(beta[j], theta[j], phi[j], nfock)
        uni = ( np.matmul(new_uni, uni ))

    return uni

In [14]:
def ecd_ansatz_np_lcu(C, beta_mat, theta_mat, phi_mat, nfock):
    nterms = beta_mat.shape[0]
    U = np.zeros((2 * nfock, 2 * nfock,), dtype=np.complex128)
    for i in range(nterms):
        U += ( C[i] * ecd_ansatz_np(beta_mat[i, :], theta_mat[i, :], phi_mat[i, :], nfock) )
    return U

In [15]:
def ecd_circuits_np_lcu(beta_mat, theta_mat, phi_mat, nfock):
    nterms = beta_mat.shape[0]
    Uten = np.zeros((nterms, 2 * nfock, 2 * nfock,), dtype=np.complex128)
    for i in range(nterms):
        Uten[i] = ecd_ansatz_np(beta_mat[i, :], theta_mat[i, :], phi_mat[i, :], nfock)
    return Uten

## Parameter organization

In [16]:
def unpack_params_np(X):
    # Initialize
    dim = X.shape[0] // 4

    # Unpack
    beta_mag = X[:dim].copy()
    beta_arg = X[dim:2*dim].copy()
    theta = X[2*dim:3*dim].copy()
    phi = X[3*dim:4*dim].copy()

    return beta_mag, beta_arg, theta, phi

In [17]:
def pack_params_np(beta_mag, beta_arg, theta, phi):
    # Initialize
    dim = beta_mag.shape[0]
    X = np.zeros((4 * dim,))

    # Pack
    X[:dim] = beta_mag.copy()
    X[dim:2*dim] = beta_arg.copy()
    X[2*dim:3*dim] = theta.copy()
    X[3*dim:4*dim] = phi.copy()

    return X

In [18]:
def unpack_params_np_lcu(X, nterms, ndepth):
    # Check
    if X.shape[0] != ( (4 * ndepth) + 1 ) * nterms:
        raise ValueError("Wrong dimensions in unpack_params_np_lcu.")

    # Initialize
    dim = 4 * ndepth
    beta_mag = np.zeros((nterms, ndepth,))
    beta_arg = np.zeros((nterms, ndepth,))
    theta = np.zeros((nterms, ndepth,))
    phi = np.zeros((nterms, ndepth,))

    # Unpack
    beta_mag[0, :], beta_arg[0, :], theta[0, :], phi[0, :] = unpack_params_np(X[:dim])
    for i in range(1, nterms):
        beta_mag[i, :], beta_arg[i, :], theta[i, :], phi[i, :] = unpack_params_np(X[i*dim:(i+1)*dim])
    C = X[-nterms:].copy()

    return C, beta_mag, beta_arg, theta, phi

In [19]:
def pack_params_np_lcu(C, beta_mag, beta_arg, theta, phi):
    # Initialize
    nterms = beta_mag.shape[0]
    ndepth = beta_mag.shape[1]
    dim = 4 * ndepth
    X = np.zeros(( (dim + 1) * nterms,))

    # Pack
    X[:dim] = pack_params_np(beta_mag[0, :], beta_arg[0, :], theta[0, :], phi[0, :])
    for i in range(nterms):
        X[i*dim:(i+1)*dim] = pack_params_np(beta_mag[i, :], beta_arg[i, :], theta[i, :], phi[i, :])
    X[-nterms:] = C.copy()

    return X

## Loss functions

In [20]:
def transition_val_np(op, n, m):
    """
    Compute <n| O |m>.

    Arguments:
    op -- Operator matrix
    n, m -- Fock basis states
    """
    # Check
    dim = op.shape[0]
    if n > dim:
        raise ValueError("n > operator dimension.")
    if m > dim:
        raise ValueError("m > operator dimension.")

    # |n> and |m>
    state1 = basis_np(dim, n)
    state2 = basis_np(dim, m)

    # <q1, n| O |q2, m>
    t1 = np.matmul(op, state2)
    ov = np.dot(np.conj(state1).T, t1)

    return np.squeeze(ov)

Let us denote $V_T$ as our target qumode matrix and $U$ as a linear combination of ECD-rotation unitaries

$$ U (\overrightarrow{\lambda}, \overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})
= \sum_j \: \lambda_j \: U_j (\overrightarrow{\beta_j}, \overrightarrow{\theta_j}, \overrightarrow{\phi_j}).
$$

Let us now explore two cost functions:

\begin{align*}
C_{diag} (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})
&= \frac{1}{2L} \: \sum_{n = 0}^{2L - 1} \:
| \langle n| ( I \otimes V_T ) |n \rangle
- \langle n| U (\overrightarrow{\lambda}, \overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})
| n \rangle |^2,
\\
C_{full} (\overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})
&= \frac{1}{4 L^2} \: \sum_{n, m = 0}^{2L - 1} \:
| \langle n| ( I \otimes V_T ) | m \rangle
- \langle n| U (\overrightarrow{\lambda}, \overrightarrow{\beta}, \overrightarrow{\theta}, \overrightarrow{\phi})
|m \rangle |^2,
\end{align*}

where $L$ is the qumode Fock cutoff.

In [21]:
def cost_fun_diag_lcu(Xvec, nterms, ndepth, V, nfock):
    """
    Loss function.

    Arguments:
    Xvec -- ECD-rotation parameters
    nterms -- number of LCU terms
    ndepth -- depth pf each ECD-rotation circuit
    V -- target qubit-qumode operator
    nfock -- Fock cutoff for qumode
    """
    # Parameters
    C, beta_mag, beta_arg, theta, phi = unpack_params_np_lcu(Xvec, nterms, ndepth)
    beta = get_cvec_np(beta_mag, beta_arg)

    # ECD unitary
    U = ecd_ansatz_np_lcu(C, beta, theta, phi, nfock)

    # Loop
    ov = 0.0
    for j in range(2 * nfock):
            v0 = transition_val_np(V, j, j)
            u0 = transition_val_np(U, j, j)
            ov += np.abs( v0 - u0 )**2

    return ov / (2 * nfock)

In [22]:
def cost_fun_full_lcu(Xvec, nterms, ndepth, V, nfock):
    """
    Loss function.

    Arguments:
    Xvec -- ECD-rotation parameters
    nterms -- number of LCU terms
    ndepth -- depth pf each ECD-rotation circuit
    V -- target qubit-qumode operator
    nfock -- Fock cutoff for qumode
    """
    # Parameters
    C, beta_mag, beta_arg, theta, phi = unpack_params_np_lcu(Xvec, nterms, ndepth)
    beta = get_cvec_np(beta_mag, beta_arg)

    # ECD unitary
    U = ecd_ansatz_np_lcu(C, beta, theta, phi, nfock)

    # Loop
    ov = 0.0
    for j in range(2 * nfock):
        for k in range(2 * nfock):
            t0 = transition_val_np(V, j, k)
            t1 = transition_val_np(U, j, k)
            ov += np.abs( t0 - t1 )**2

    return ov / (4 * nfock**2)

## Expectation value

### Tracing out

We will use `QuTip` for this part.

In [23]:
def trace_out_qubit(state, nfock):
    """
    Performs a partial trace over the qubit part of a density matrix,
    leaving the reduced density matrix for the qumode.

    Arguments:
    state -- statevector for qubit-qumode system
    nfock -- Fock cutoff
    """
    # Initialize
    if not isinstance(state, qt.Qobj):
        qstate = qt.Qobj(state)
    else:
        qstate = state

    # Dimension matching
    dims = [[2, nfock], [1, 1]]
    tstate = qt.Qobj(qstate.data, dims=dims)

    # Get density matrix
    rho_qumode = tstate.ptrace(1)

    return rho_qumode

In [24]:
def trace_out_qumode(state, nfock):
    """
    Performs a partial trace over the qumode part of a density matrix,
    leaving the reduced density matrix for the qubit.

    Arguments:
    state -- statevector for qubit-qumode system
    nfock -- Fock cutoff
    """
    # Initialize
    if not isinstance(state, qt.Qobj):
        qstate = qt.Qobj(state)
    else:
        qstate = state

    # Dimension matching
    dims = [[2, nfock], [1, 1]]
    tstate = qt.Qobj(qstate.data, dims=dims)

    # Get density matrix
    rho_qubit = tstate.ptrace(0)

    return rho_qubit

### Hadamard test

In [25]:
def measure_qubit(state, nfock):
    """
    Given a qubit-qumode state, compute the expectation value
    of Z by only measuring the qubit.
    """
    rho = trace_out_qumode(state, nfock)

    return qt.expect(qt.sigmaz(), rho)

In [26]:
def include_hadamard(U):
    """
    Build the full qubit-qumode operator for Hadamard test
    on the qubit with qumode operator U.
    """
    nfock = U.shape[0]
    V1 = np.kron(hadamard_np(), identity_np(nfock))
    V2 = np.kron(qproj00_np().astype(np.complex128), identity_np(nfock))
    V2 += np.kron(qproj11_np().astype(np.complex128), U)

    V3 = np.matmul(V2, V1)
    V = np.matmul(V1, V3)

    return V

In [27]:
def qubit_measure_random_lcu(V, Uten, C, nsample=5):
    """
    Can we get Re(<psi | V | psi>) by Hadamard matrices M(j)?
    Here, we have a linear combination of U(j) matrices, each of
    which constructs the M(j) matrices.

    V -- reference qumode matrix
    Uten -- test sets of qubit-qumode unitaries
    C -- LCU coefficients
    """
    # Initialize
    S1 = np.zeros((nsample,))
    S2 = np.zeros((nsample,))

    # Target operator
    Vop = qt.Qobj( np.kron(identity_np(2), V) )

    # Hadamard blocks
    nterms = Uten.shape[0]
    nfock = Uten.shape[1] // 2
    Mten = np.zeros((nterms, 4 * nfock, 4 * nfock,), dtype=np.complex128)
    for i in range(nterms):
        Mten[i, :, :] = include_hadamard(Uten[i, :, :])

    # Loop
    for i in range(nsample):
        # |Psi> = |0, psi>
        cstate = qt.rand_ket(2 * nfock).unit()
        fstate = qt.tensor(qt.basis(2, 0), cstate).full()

        # EV1
        S1[i] = 0.0
        for j in range(nterms):
            # |Psi'> = M(j) |Psi>
            psi = np.matmul(Mten[j, :, :], fstate)

            # Measure
            S1[i] += ( C[j] * measure_qubit(psi, 2 * nfock) )

        # EV2
        S2[i] = np.real(qt.expect(Vop, cstate))

    return S1, S2

## Core optimizations

### Regular

In [28]:
def unpack_fun_np_lcu(X, nterms, ndepth, V, nfock, cf):
    if cf == 'diag':
        return cost_fun_diag_lcu(X, nterms, ndepth, V, nfock)
    elif cf == 'full':
        return cost_fun_full_lcu(X, nterms, ndepth, V, nfock)
    else:
        print(f"Error: cf not found")
        sys.exit(1)

In [29]:
def opt_sckit_lcu(V, ndepth, nterms, cf, budget=100, method='imfil', Xvec=[]):
    """
    Minimize the cost function using Scikit-based methods.

    Arguments:
    V -- target qumode matrix
    ndepth -- ansatz circuit depth
    budget -- maximum number of iterations
    method -- optimization method
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    # Fock cutoff
    nfock = V.shape[0]
    FullV = np.kron(identity_np(2), V)

    # Bound parameters
    C_min = -2.0
    C_max = 2.0
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    bounds = []
    for j in range(nterms):
        for _ in range(ndepth):
            bounds.append([beta_mag_min, beta_mag_max])
        for _ in range(ndepth):
            bounds.append([beta_arg_min, beta_arg_max])
        for _ in range(ndepth):
            bounds.append([theta_min, theta_max])
        for _ in range(ndepth):
            bounds.append([phi_min, phi_max])
    for j in range(nterms):
        bounds.append([C_min, C_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        C = np.random.uniform(-0.1, 0.1, size=nterms)
        beta_mag = np.random.uniform(0, 3, size=(nterms, ndepth))
        beta_arg = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        theta = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        phi = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        Xvec = pack_params_np_lcu(C, beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(unpack_fun_np_lcu, nterms=nterms, ndepth=ndepth, V=FullV, nfock=nfock, cf=cf)

    # Optimize
    if method == 'bobyqa':
        result, history = skopt.minimize(obj_fun, Xvec, bounds, budget, method='bobyqa')
    else:
        result, history = skopt.minimize(obj_fun, Xvec, bounds, budget, method='imfil')

    return result.optval, result.optpar

In [30]:
def opt_scipy_lcu(V, ndepth, nterms, cf, maxiter=100, method='COBYLA', \
                  threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using SciPy-based methods.

    Arguments:
    V -- target qumode matrix
    ndepth -- ansatz circuit depth
    maxiter -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    # Fock cutoff
    nfock = V.shape[0]
    FullV = np.kron(identity_np(2), V)

    # Bound parameters
    C_min = -2.0
    C_max = 2.0
    beta_mag_min = 0.0
    beta_mag_max = 10.0
    beta_arg_min = 0.0
    beta_arg_max = 2 * np.pi
    theta_min = 0.0
    theta_max = np.pi
    phi_min = 0.0
    phi_max = 2 * np.pi

    # Define bounds
    bounds = []
    for j in range(nterms):
        for _ in range(ndepth):
            bounds.append([beta_mag_min, beta_mag_max])
        for _ in range(ndepth):
            bounds.append([beta_arg_min, beta_arg_max])
        for _ in range(ndepth):
            bounds.append([theta_min, theta_max])
        for _ in range(ndepth):
            bounds.append([phi_min, phi_max])
    for j in range(nterms):
        bounds.append([C_min, C_max])
    bounds = np.array(bounds)

    # Guess
    if len(Xvec) == 0:
        C = np.random.uniform(-0.1, 0.1, size=nterms)
        beta_mag = np.random.uniform(0, 3, size=(nterms, ndepth))
        beta_arg = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        theta = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        phi = np.random.uniform(0, np.pi, size=(nterms, ndepth))
        Xvec = pack_params_np_lcu(C, beta_mag, beta_arg, theta, phi)

    # Loss function
    obj_fun = partial(unpack_fun_np_lcu, nterms=nterms, ndepth=ndepth, V=FullV, nfock=nfock, cf=cf)

    # SciPy options
    options = {'disp': True, 'maxiter': maxiter}

    # Optimize
    if method == 'COBYLA':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options)
    elif method == 'Powell':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options)
    elif method == 'CG':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options)
    elif method == 'BFGS':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 tol=threshold, options=options)
    elif method == 'L-BFGS-B':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options)
    elif method == 'TNC':
        result = sciopt.minimize(obj_fun, Xvec, method=method, \
                                 bounds=bounds, tol=threshold)
    elif method == 'trust-constr':
        result = sciopt.minimize(obj_fun, Xvec, method=method, bounds=bounds, \
                                 tol=threshold, options=options)
    else:
        result = sciopt.minimize(obj_fun, Xvec, method='COBYLA', bounds=bounds, \
                                 tol=threshold, options=options)

    return result.fun, result.x

In [31]:
def ecd_opt_lcu_regular(V, ndepth, nterms, cf, budget=100, method='imfil', \
                        threshold=1e-08, Xvec=[]):
    """
    Minimize the cost function using various optimizers.

    Arguments:
    V -- target qumode matrix
    ndepth -- ansatz circuit depth
    budget -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    beta_mag, beta_arg, theta, phi -- optional initial guesses
    """
    # Method library
    sk_tuple = ('imfil', 'bobyqa',)
    sc_tuple = ('COBYLA', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'trust-constr')

    # Optimize
    if method in sk_tuple:
        loss, Xvec = opt_sckit_lcu(V, ndepth, nterms, cf=cf, budget=budget, \
                                   method=method, Xvec=Xvec)
    elif method in sc_tuple:
        loss, Xvec = opt_scipy_lcu(V, ndepth, nterms, cf=cf, maxiter=budget, \
                                   method=method, threshold=threshold, Xvec=Xvec)
    else:
        print(f"Error: method not found")
        sys.exit(1)

    return loss, Xvec

### Multi-start

In [32]:
def ecd_opt_lcu_sample(V, ndepth, nterms, cf, ns=10, verb=1, budget=100, \
                       method='imfil', threshold=1e-08):
    """
    Minimize the cost function with different sets of initial guesses.

    Arguments:
    V -- target qumode matrix
    ndepth -- ansatz circuit depth
    ns -- number of guess sets
    verb -- decide additional printing
    budget -- maximum number of iterations
    method -- optimization method
    threshold -- error tolerance
    """
    # Initialize
    prev_loss = float('inf')
    dim = ( (4 * ndepth) + 1 ) * nterms
    Xvec = np.zeros((dim,))

    # Print
    if verb == 1:
        print("========Sample========")

    # Sampling loop
    for j in range(ns):

        # Optimize
        loss, X_f = ecd_opt_lcu_regular(V, ndepth, nterms, cf, budget=budget, \
                                        method=method, threshold=threshold)

        # Print
        if verb == 1:
            print("--------")
            print("sample number: ", j+1)
            print("loss: ", loss)

        # Choose
        if loss < prev_loss:
            prev_loss = loss
            Xvec = X_f

    return prev_loss, Xvec

## Explore

In [ ]:
def test_core_lcu_uni(ndepth, nterms):
    # Target
    V = qt.tensor( qt.sigmax(), qt.qeye(4) )
    V = V.full()

    # Optimize
    for i in range(nterms):
        loss, Xvec = ecd_opt_lcu_regular(V, ndepth, i+1, 'full', budget=int(2e3), \
                                         method='BFGS', threshold=1e-12)
        print(f"loss of {i+1} terms: {loss}")

    return loss

In [ ]:
ndepth = 4
nterms = 10

test_core_lcu_uni(ndepth, nterms)

In [ ]:
def test_core_lcu_hadamard(ndepth, nterms, nsample):
    # Target
    V = qt.sigmax() #qt.tensor( qt.sigmax(), qt.sigmaz() )
    V = V.full()

    # Optimize
    loss, Xvec = ecd_opt_lcu_sample(V, ndepth, nterms, 'full', ns=1, verb=1, \
                                    budget=int(2e2), method='BFGS', threshold=1e-12)

    # Print
    print("----------")
    print("Loss: ", loss)

    # ECD unitary
    nfock = V.shape[0]
    C, beta_mag, beta_arg, theta, phi = unpack_params_np_lcu(Xvec, nterms, ndepth)
    beta = get_cvec_np(beta_mag, beta_arg)
    Uten = ecd_circuits_np_lcu(beta, theta, phi, nfock)

    # Hadamard test
    S1, S2 = qubit_measure_random_lcu(V, Uten, C, nsample=5)

    return np.abs(S1 - S2)

In [ ]:
ndepth = 2
nterms = 1
nsample = 5

test_core_lcu_hadamard(ndepth, nterms, nsample)